# Problema do Transporte

## Importando Bibliotecas

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

## Dados

In [10]:
# Dados Entrada

i = ['seattle', 'san_diego']
j = ['new_york','chicago','topeka']

cap = [350,600]
dem = [325,300,275]

distancias = pd.DataFrame([[2.5,1.7,1.8],[2.5,1.8,1.4]], columns = j, index = i) # em 1000 km

preco_km = 90 # preco de transporte a cada 1000 km

custo = (preco_km * distancias)/1000   # custo de transporte por km, entre origens e destinos

## Modelo no Pyomo

In [11]:
ni = len(i)
nj = len(j)

model = ConcreteModel()

#Sets:
model.i = Set(initialize=i)
model.j = Set(initialize=j)

#Parâmetros
model.cap = Param(model.i, initialize={i[k]: cap[k] for k in range(ni)})
model.dem = Param(model.j, initialize={j[k]: dem[k] for k in range(nj)})
model.distancias = Param(model.i, model.j, initialize=lambda model, i, j: distancias.loc[i,j])
model.custo = Param(model.i, model.j, initialize=lambda model, i, j: custo.loc[i,j])

#Variáveis
model.x = Var(model.i, model.j, domain=NonNegativeReals)

#Declarar Função Objetivo
def objective_rule(model):
    return sum(model.custo[i,j]*model.x[i,j] for i in model.i for j in model.j)
model.objective = Objective(rule=objective_rule, sense=minimize)

#Declarar Restrições
def supply_rule(model,i):
    return sum(model.x[i,j] for j in model.j) <= model.cap[i]
model.supply = Constraint(model.i, rule=supply_rule)

def demand_rule(model,j):
    return sum(model.x[i,j] for i in model.i) >= model.dem[j]
model.demand = Constraint(model.j, rule=demand_rule)

#Executar Otimização
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

## Resultados

In [12]:
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo:', value(model.objective), '\n')

for orig in model.i:
    for dest in model.j:
        print('Produtos transportados de {} para {}:'.format(orig, dest), model.x[orig, dest].value)

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: 153.67499999999998 

Produtos transportados de seattle para new_york: 50.0
Produtos transportados de seattle para chicago: 300.0
Produtos transportados de seattle para topeka: 0.0
Produtos transportados de san_diego para new_york: 275.0
Produtos transportados de san_diego para chicago: 0.0
Produtos transportados de san_diego para topeka: 275.0
